In [1]:
import pandas as pd
# run this command to allow more data to be displayed than default
pd.set_option('display.max_rows', 200)

In [2]:
#read in the data (this is the dis2.zip file from Maarten)
travel_bus = pd.read_csv("Data/dis2.csv")
#Check that it's been read in 
travel_bus

,origin_i,origin_msoacode,destination_j,destination_msoacode,data
0,0,E02000001,0,E02000001,6.483222
1,0,E02000001,1,E02000002,67.573280
2,0,E02000001,2,E02000003,59.130450
3,0,E02000001,3,E02000004,65.570350
4,0,E02000001,4,E02000005,57.211044
...,...,...,...,...,...
71166091,8435,S02001235,8431,S02001231,26.515114
71166092,8435,S02001235,8432,S02001232,3.424564
71166093,8435,S02001235,8433,S02001233,20.985640
71166094,8435,S02001235,8434,S02001234,6.020671


In [3]:
# drop the origin_i and destination_j columns
travel_bus.drop(['origin_i', 'destination_j'], axis=1, inplace=True)
# rename the data column 
travel_bus.rename({'data': 'time_bus_min'}, axis=1, inplace=True)

The data column is travel time between MSOAs in minutes. We want to get:
1 - UNWEIGHTED Average Travel Time From MSOA to all Other MSOAs (by mode)
2 - WEIGHTED Average Travel Time From MSOA to all Other MSOAs (by mode)

1 - UNWEIGHTED Average Travel Time From MSOA to all Other MSOAs (by mode)

In [4]:
#merge by origin MSOA and average over all transport options
avg_bus = travel_bus.groupby("origin_msoacode").mean()
# rename the time_bus_min column 
avg_bus.rename({'time_bus_min': 'avg_time_from_origin_bus_UNWEIGHTED'}, axis=1, inplace=True)
avg_bus

,avg_time_from_origin_bus_UNWEIGHTED
origin_msoacode,
E02000001,304.550522
E02000002,349.848830
E02000003,349.557987
E02000004,353.195913
E02000005,350.006653
...,...
W02000419,406.200740
W02000420,352.533895
W02000421,411.257256


2 - WEIGHTED Average Travel Time From MSOA to all Other MSOAs (by mode)

In [5]:
#import the flow data (from here https://www.nomisweb.co.uk/census/2011/bulk/rOD1)
flow = pd.read_csv("Data/wu03ew_msoa.csv")
flow

,Area of residence,Area of workplace,All categories: Method of travel to work,Work mainly at or from home,"Underground, metro, light rail, tram",Train,"Bus, minibus or coach",Taxi,"Motorcycle, scooter or moped",Driving a car or van,Passenger in a car or van,Bicycle,On foot,Other method of travel to work
0,E02000001,E02000001,1506,0,73,41,32,9,1,8,1,33,1304,4
1,E02000001,E02000014,2,0,2,0,0,0,0,0,0,0,0,0
2,E02000001,E02000016,3,0,1,0,2,0,0,0,0,0,0,0
3,E02000001,E02000025,1,0,0,1,0,0,0,0,0,0,0,0
4,E02000001,E02000028,1,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2402196,W02000423,W02000411,6,0,0,0,3,0,0,1,0,0,2,0
2402197,W02000423,W02000412,58,0,0,0,10,0,1,29,1,4,13,0
2402198,W02000423,W02000415,3,0,0,0,0,0,0,3,0,0,0,0
2402199,W02000423,W02000422,525,0,1,2,17,2,0,125,11,31,333,3


In [6]:
# keep only the bus column
flow = flow[['Area of residence','Area of workplace', 'Bus, minibus or coach']]
flow

,Area of residence,Area of workplace,"Bus, minibus or coach"
0,E02000001,E02000001,32
1,E02000001,E02000014,0
2,E02000001,E02000016,2
3,E02000001,E02000025,0
4,E02000001,E02000028,0
...,...,...,...
2402196,W02000423,W02000411,3
2402197,W02000423,W02000412,10
2402198,W02000423,W02000415,0
2402199,W02000423,W02000422,17


In [7]:
# merge the df with the travel times to the flow df. The condition is that thae origin MSOA 
# and the Destination MSOA are the same
flow_distance = pd.merge(travel_bus, flow,  how='left', 
                         left_on=['origin_msoacode','destination_msoacode'], 
                         right_on = ['Area of residence','Area of workplace'])
flow_distance.head(5)

,origin_msoacode,destination_msoacode,time_bus_min,Area of residence,Area of workplace,"Bus, minibus or coach"
0,E02000001,E02000001,6.483222,E02000001,E02000001,32.0
1,E02000001,E02000002,67.573280,NaN,NaN,NaN
2,E02000001,E02000003,59.130450,NaN,NaN,NaN
3,E02000001,E02000004,65.570350,NaN,NaN,NaN
4,E02000001,E02000005,57.211044,NaN,NaN,NaN


In [8]:
#replace all Nan values in the last column with 0
flow_distance['Bus, minibus or coach'].fillna(0, inplace=True)
# check 
flow_distance.head(5)

,origin_msoacode,destination_msoacode,time_bus_min,Area of residence,Area of workplace,"Bus, minibus or coach"
0,E02000001,E02000001,6.483222,E02000001,E02000001,32.0
1,E02000001,E02000002,67.573280,NaN,NaN,0.0
2,E02000001,E02000003,59.130450,NaN,NaN,0.0
3,E02000001,E02000004,65.570350,NaN,NaN,0.0
4,E02000001,E02000005,57.211044,NaN,NaN,0.0


In [9]:
# create a new column with the weighted travel time
flow_distance['cumulative_time_bus'] = flow_distance['time_bus_min'] * flow_distance['Bus, minibus or coach']
#check
flow_distance.head(5)

,origin_msoacode,destination_msoacode,time_bus_min,Area of residence,Area of workplace,"Bus, minibus or coach",cumulative_time_bus
0,E02000001,E02000001,6.483222,E02000001,E02000001,32.0,207.463104
1,E02000001,E02000002,67.573280,NaN,NaN,0.0,0.000000
2,E02000001,E02000003,59.130450,NaN,NaN,0.0,0.000000
3,E02000001,E02000004,65.570350,NaN,NaN,0.0,0.000000
4,E02000001,E02000005,57.211044,NaN,NaN,0.0,0.000000


In [10]:
# group by origin MSOA and get the sum
avg_bus_weighted = flow_distance.groupby("origin_msoacode").sum()
avg_bus_weighted.head(5)

,time_bus_min,"Bus, minibus or coach",cumulative_time_bus
origin_msoacode,,,
E02000001,2.569188e+06,252.0,4731.529671
E02000002,2.951325e+06,330.0,9989.492729
E02000003,2.948871e+06,442.0,13493.194688
E02000004,2.979561e+06,278.0,7004.190858
E02000005,2.952656e+06,500.0,14035.449835


In [11]:
# add column to get average travel time by bus
avg_bus_weighted['avg_time_bus'] = avg_bus_weighted['cumulative_time_bus'] / avg_bus_weighted['Bus, minibus or coach']
avg_bus_weighted.head(5)

,time_bus_min,"Bus, minibus or coach",cumulative_time_bus,avg_time_bus
origin_msoacode,,,,
E02000001,2.569188e+06,252.0,4731.529671,18.775911
E02000002,2.951325e+06,330.0,9989.492729,30.271190
E02000003,2.948871e+06,442.0,13493.194688,30.527590
E02000004,2.979561e+06,278.0,7004.190858,25.194931
E02000005,2.952656e+06,500.0,14035.449835,28.070900


In [12]:
#merge the avg travel time df with this df 

bus = pd.merge(avg_bus, avg_bus_weighted,  how='left', 
                         left_on=['origin_msoacode'], 
                         right_on = ['origin_msoacode'])
bus

,avg_time_from_origin_bus_UNWEIGHTED,time_bus_min,"Bus, minibus or coach",cumulative_time_bus,avg_time_bus
origin_msoacode,,,,,
E02000001,304.550522,2.569188e+06,252.0,4731.529671,18.775911
E02000002,349.848830,2.951325e+06,330.0,9989.492729,30.271190
E02000003,349.557987,2.948871e+06,442.0,13493.194688,30.527590
E02000004,353.195913,2.979561e+06,278.0,7004.190858,25.194931
E02000005,350.006653,2.952656e+06,500.0,14035.449835,28.070900
...,...,...,...,...,...
W02000419,406.200740,3.426709e+06,153.0,2963.981591,19.372429
W02000420,352.533895,2.973976e+06,238.0,5810.227354,24.412720
W02000421,411.257256,3.469366e+06,96.0,3756.583521,39.131078


In [13]:
# drop time_bus_min: we don't want a sum of the time from each MSOA to all other MSOAs
bus.drop('time_bus_min', axis=1, inplace=True)
bus

,avg_time_from_origin_bus_UNWEIGHTED,"Bus, minibus or coach",cumulative_time_bus,avg_time_bus
origin_msoacode,,,,
E02000001,304.550522,252.0,4731.529671,18.775911
E02000002,349.848830,330.0,9989.492729,30.271190
E02000003,349.557987,442.0,13493.194688,30.527590
E02000004,353.195913,278.0,7004.190858,25.194931
E02000005,350.006653,500.0,14035.449835,28.070900
...,...,...,...,...
W02000419,406.200740,153.0,2963.981591,19.372429
W02000420,352.533895,238.0,5810.227354,24.412720
W02000421,411.257256,96.0,3756.583521,39.131078


In [14]:
#save to csv
bus.to_csv('travel_time_bus.csv')